# Chapter 7: Hypothesis and Inference

## Statistical Hypothesis Testing

Hypothesis are assertions like "This coin is fair", "Data Scientists prefer python to R", "Popups dissuade visitors to our site", etc, which can be translated into statistics about data. Under various assumptions, those statistics can be thought of as observations of random variables from known distributions, which allows us to make statements about how likely those assumtpions to hold. 

In the classic setup, we have a null hypothesis ${H_0}$, that represents some default position and some alternative hypothesis ${H_1}$ that we'd like to compare it with. We use statistcs to decide whether we can reject ${H_0}$ as false or not. 

### Example- Flipping a Coin. 

Imagine we have a coin, and we want to test whether it's fair. We make the assumption that the coin has some probability $p$ of landing on heads, so our null hypothesis is that the coin is fair- that $$p = 0.5$$. We'll test this agains the alternative hypothesis $$p \neq 0.5$$. 

In particular, our test will involve flipping the coin some number $n$ times and counting the number of heads. Each coin flip is a Bernoulli trial, which means that X is a Binomial(n,p) random variable which(as we saw in Chapeter 6), we can approximate using the normal distribution. 



In [2]:
import random
import math

import ipynb.fs.defs.Chapter6_Probability as prob

In [3]:
def normal_approximation_to_binomial(n,p):
    """finds mu and sigma corresponding to a Binomial (n,p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma



We can use the normal_cdf function to figure out the probability that a random variable (following a normal distriubution) lies inside or outside a particular interval. 


In [4]:
# the normal cdf is the probability the variable is below a threshold
normal_probability_below = prob.normal_cdf          # function synonym

# Considered above the threshold if it's not below: 
def normal_probability_above(lo, mu=0, sigma=1):
    return 1- prob.normal_cdf(lo, mu, sigma)

# It's between if it's less than hi but not less than lo. 
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return prob.normal_cdf(hi, mu, sigma) - prob.normal_cdf(lo, mu, sigma)

# It's outside if it's not between
def normal_probability_outside(lo, hi, mu, sigma):
    return 1 - normal_probability_between(lo, hi, mu, sigma)


Can also do the reverse- find the non-tail region or the symmetric interval around the mean that accounts for a certain level of likelihood. For example, if we want to find an interval centred at the mean and containing 60% probability, then we find the cutoffs where the upper and lower tail each contain 20% of the probability (thereby leaving 60%). 

In [5]:
def normal_upper_bound(probability, mu=0, sigma=1):
    """return the z for which P( Z <= z ) == probability"""
    return prob.inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P( Z >= z ) == probability"""
    return prob.inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """return the symmetric (about the mean) bounds 
       that contain the specified probability """
    tail_probability = (1 - probability) / 2
    
    # Upper bound probability should have tail probability above it. 
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # Lower bound probability should have tail probability above it. 
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound



Say we wanted to flip the coin 1000 times. If the coin was fair, X should be approximately normal with mean 500 and standard deviation 15.8. 

We need to make a decision about significance- how willing we are to make a type 1 error (false positive classification), in which we reject ${H_0}$ even though it is actually true. For reasons lost to time, the significance level is usually either 5% (1 in 20 type 1 error) or 1% (1 in 100 type 1 error). Choosing 5%: 



In [6]:
mu_0, sigma_0 = normal_approximation_to_binomial( 1000, 0.5)

# Determine the test that rejects H0 if X falls outside the bounds below: 

normal_two_sided_bounds( 0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

Assuming p really equals 0.5 (i.e. ${H_0}$ is true), there is just a 5% chance we observe an X that lies outside this interval, which is the exact significance we wanted.  Said differently, if ${H_0}$ is true then, approximately 19 times out of 20, this test will give the correct result. 

We're often interested in the power of a test, which is the probability of not making a type 2 error, in which we fail to reject ${H_0}$ even though it is really false. In order to measure this, we have to specify what exactly ${H_0}$ being false means- knowing that p is not 0.5 doesn't say anything abpout the distribution of X. In particular, lets check what happens if p is really 0.55, so the coin has a slight bias towards heads. 

The power of the test can be calculated as below. 

In [7]:
# 95% bounds based on assumption p is 0.5. 
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lo, hi)

# actual mu and sigma based on p=0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

print(mu_1, sigma_1)

# A type 2 error means we fail to reject the null hypothesis (H0)
# which will happen when X is still in our original interval. 
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)

power = 1 - type_2_probability

print(power)

469.01026640487555 530.9897335951244
550.0 15.732132722552274
0.8865480012953671


Imagine instead that our null hypothesis ${H_0}$ was that the coin is not biased towards heads, or that $p \leq 0.5$. In that case, we want a one-sided test that rejects the null hypotheses when X is much larger than 500 but not when X is smaller than 500. So, a 5% significance test involves using normal_probability_below to find the cutoff below which 95% of the probabilities lie. 

In [8]:
hi= normal_upper_bound(0.95, mu_0, sigma_0)
print(hi)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

526.0073585242053
0.9363794803307173


## p-values

p-values provide a different way to think about this. Instead of looking at broundaries based on a probability cutoff, we instead compute the probability (assuming ${H_0}$ is true) that we would see a value at least as extreme as the one we actually observed. 

For our two-sided test of whether the coin is fair, we could compute: 


In [11]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu: 
        
        # if x is greater than the mean, the tail is what's greater than X...
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the mean, the tail is what's below x. 
        return 2 * normal_probability_below(x, mu, sigma)
    
# e.g. p value for seeing 530 heads
two_sided_p_value(529.5, mu_0, sigma_0)
        

0.06207721579598857

### Note

The change from 530 to 529.5 is what's called a continuity correction. 
It reflects the fact that normal_probability_between(529.5, 350.5, mu_0, sigma_0) is a better estimate of the probability of seeing 530 heads than norma_probability_between(530,531,mu_0, sigma_0). 

Correspondingly, normal_probability_above(529.5, mu_0, sigma_0) is a better estimate of the probability of seeing at least 530 heads. 

In [12]:
# Simulation of this... 

extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0    # count of heads
                   for _ in range(1000))                 # in 1000 flips
    if num_heads >= 530 or num_heads <= 470:              # count how often
        extreme_value_count += 1                         # we see extremes
        
print(extreme_value_count / 100000)

0.06155


Warning!

Make sre your data is roughly normally distributed before using normal_probability_above to compute p-values. Most predictive algorithms assume the target variable is normally distributed- if it's not, you're on shaky ground. 

there are various statistical tests to determine whether a variable is normally distributed, but even plotting the data is a good start.

# Confidence Intervals

We have been testing hypotheses about the value of the heads probability p, which is the parameter of the unknown "heads" distribution. When this is the case, a third approach is to construct a confidence interval around the observed value of the parameter. 

For example, we can estimate the probability of the unfair coin by looking at the average value of the Bernoulli variables corresponding to each flip- 1 if heds, 0 if tails. If we observe 525 heads out of 1000 flips, then we estimate p equals 0.525. 

How confident can we be about this estimate? Well. if we knew the exact value of p, the central limit theorem tells us that the average of those Bernoulli variables should be approximately normal with mean p and standard deviation 

In [13]:
math.sqrt(p * (1 - P) /1000)

NameError: name 'p' is not defined

Here, we don't know p, so instead we use our estimate. 

In [15]:
p_hat = 525 / 1000
mu = p_hat

sigma= math.sqrt(p_hat * (1-p_hat)/ 1000)
print(sigma)

0.015791611697353755


This is not entirely justified, but people seem to do it anyway. Using the normal approximation, we conclude that we are "95%" confident that the following interval contains the true parameter p. 



In [16]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

Note
This is a statement abot the interval- not the parameter p. You should read it as the assertion that if you were to repeat the experiment many times, 95% of the time the "true" parameter (which is the same every time) would lie within the observed confidence interval, which may be different every time. 


In particular, we do not conclude that the coin is unfair, since 0.5 falls within our confidence interval. 
If instead, we'd seen 540 heads, then we'd have: 

In [17]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1-p_hat) / 1000)

normal_two_sided_bounds(0.95, mu, sigma)

(0.5091095927295919, 0.5708904072704082)

The fair coin parameter (p = 0.5) doesn't lie within this interval. The fair coin hypothesis doesn't pass a test that you'd expect it to pass 95% of the time if it were true. '

# P-Hacking

A procedure that erroneously rejects the null hypothesis only 5% of the time will (by definition) 5% of the time erroneously reject the null hypothesis. 

In [18]:
def run_experiment():
    """flip a fair coin 1000 times. True = heads, false = tails."""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """ using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                     for experiment in experiments
                     if reject_fairness(experiment)])

print(num_rejections)


46


What this means is, if you're setting out to find "significant" results, you usually can. Test enough hypotheses against your data set and one of them will almost certainly appear significant. Remove the right outliers and you can probably get your p-value below 0.05. 
This is sometimes called p-hacking and is in some ways a consequence of the "inference from p-values framework". 
There are issues with this approach- see article "The earth is round". 

If you want to do good science, you should determine your hypothesis before looking at the datam you should clean the data without the hypotheses in mind and you should keep in mind that p-values are not a substitute for common sense. 
An alternative approach is Bayesian Inference. 

# Example: Running an A/B Test

One of your primary responsibilities at DataSciencester is experience optimisation, which is a euphemism for trying to get people to click on adverts. 
One of your advertisers has developed a new energy drink targeted at data scientists and the VP of Advertisments wants your help choosing between advertisment A ("tastes great!") and advertisment B ("less bias!"). 

Being a scientist, you decide to run an experiment by randomly showing each site visitor one of the two advertisments and tracking how many people click on each one. 

If 990 out of 1000 A-viewers click their add while only 10 out of 1000 B-viewers click their ad, you can be pretty confident that A is the better ad. But, what if the differences are not so stark? Here's where you'd use statistical inference. 

Lets say that $N_A$ people see ad A, and that $n_A$ of them click it. We can think of each Bernoulli trial where $p_A$ is the probability that someonbe clicks ad A. Then, if $N_A$ is large, which it is here, we know that $n_A / N_A$ is approximately a normal random variable with mean $p_A$ and standard deviation $\sigma_A = \sqrt{p_A(1-p_A) / N_A}$

Similarly, $n_B / N/B$ is approximately a normal random variable with mean $p_B$ and standard deviation $\sigma_B = \sqrt{p_B (1 - p_B) / N_B}$

In [46]:
def estimated_parameters(N,n):
    p = n / N
    sigma = math.sqrt( ( p * (1.0 - p) / N) )
    return p, sigma


If we assume those two normals are independent (which would seem reasonable, since the individual Bernoulli trials should be), then their difference should also be normal with mean $p_B - p_A$ and standard deviation $\sqrt{\sigma^2_A + \sigma^2_B}$

# Note
This is a type of cheating. The maths only works out exactly like this if you know the standard deviations. HEre, we're estimating them from the data, which means we really should be using the t-distribution. But, for a large enough data set, it's close enough that it doesn't make much difference. 

This means we can test the *null hypothesis* that $p_A $ and $p_B$ are the same (that is, $p_A - p_B$ is zero using the statistic: 



In [44]:
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(pow(sigma_A,2.0) + pow(sigma_B, 2.0))

This should approximately be a standard nbormal variable. 
For example, if "tastes great" gets 200 clicks from 1000 views and "less bias" gets 180 clicks out of 1000 views, the statistic is: 

In [47]:
z = a_b_test_statistic(1000, 200, 1000, 180)

print(z)

-1.1403464899034472


The probability of seeing such a large difference if the means were actually equal would be: 

In [48]:
two_sided_p_value(z)

0.254141976542236

which is large enough that you can't conclude there is much of a difference. On the other hand, if "less bias" got 150 clicks, we'd have: 

In [50]:
z = a_b_test_statistic(1000, 200, 1000, 150)
two_sided_p_value(z)

0.003189699706216853

## Bayesian Inference

The procedures we've looked at have involved making probability statements about our tests- "there's only a 4% chance you'd observe such an extreme statistic if our null hypothesis were true. 

An alternative apporach to inference involves treating the unknown parameters themselves as random variables. The analyst (that's you) starts with a *prior distribution* for the parameters and then use the observed data and Bayes Theorem to get an updated *posterior distribution* for the parameters. Rather than making probability judgments about the tests, you make probability judgments about the parameters themselves. 

For example, when the unknown parameter is a probability (as in our coin flipping example), we often use a prior from the *beta distribution*, which puts all its probability between 0 and 1. 

                                                                                                                           

In [ ]:
def B(alpha, beta):
    """a normalising constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x <= 0 or x >= 1:           # no weight outside of [0,1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)


Generally speaking, this distribution centres its weight at: 
    
$alpha / (alpha + beta)$

and the larger alpha and beta are, the tighter the distribution is. 

For example, if alpha and beta are both 1, it's just the uniform distribution (centred at 0.5, very dispersed) If alpha is much larger than beta, most of the weight is near 1. And if alpha is much smaller than beta, most of the weight is near zero. 

So, lets say we assume a prior distribution on $p$. Maybe we don;t want to take a stand on whether the coin is fair and we choose alpha and beta to both equal 1. Or maybe we have a strong belief that it lands heads 55% of the time and we choose alpha equals 44, beta equals 45. 

Then we flip our coin a bunch of times and see $h$ heads and $t$ tails. Bayes Theorem (and some mathematics that's too tedious to go through here) tells us tht the posterior distribution for p is again a Beta distribution but with parameters alpha + h and beta + t. 




### Note

It is no coincidence that the posterior distribution was again a Beta distribution. The number of heads is given by a Binomial distribution and the Beta is the *conjugate prior* to the Binomial distribution. This means that whenever you update a Beta prior using observations from the corresponding binomial, you will get back a Beta posterior. 



Lets say you flip the coin 10 times and see only 3 heads. 

If you started with the uniform prior (in some sense refusing to take a stand about the coin's fairness), your posterior distribution would be Beta(4,8) centres around 0.33. Since you considered all probabilities equally likely, your best guess is something pretty close to the obserbed probability. 

If you started with a Beta(20,20) expressing the belief that the coin was approximately fair, your posterior distribution would be a Beta(23,27), centred around 0.46 indicating a revised belief that maybe the coin is slightly biased towards tails. 

And if you started with a Beta(30,10) expressing a belief that the coin was biased to flip 75% heads, your posterior distribution would be a Beta(33,17) cenred around 0.66. In that case you'd still believe in a heads bias but less strongly than you did initially. 

If you flipped the coin more and more times, the prior would matter less and less until eventually you'd have (nearly) the same posterior distribution no mateer which prior you started with. 

For example, no matter how biased you initially thought the coin was, it would be hard to maintain that belief after seeing 1000 heads out of 2000 flips (unless you are a lunatic who picks something like a Beta(1000000,1) prior. 

What's interesting is that this allows us to make probability statements about hypotheses: "Based on the prior and observed data, there is only a 5% likelihood he coin's heads probability is between 49% and 51%". 

This is philosophically different from a statement like: "If the coin were fair, we'd expect to observe data so extreme only 5% of the time". 

Using Bayesian inference to test hypotheses is considered somewhat controversial, in part because its mathematics can get somewhat complicated and in part because  of the subjective nature of choosing a prior. We won't cover it any further, but it's good to be aware of. 

